In [3]:
import pandas as pd
import numpy as np

In [63]:
df_listing = pd.read_csv('listings 2 reduced.csv')
df_review = pd.read_csv('reviews 2 reduced.csv')

C:\Users\vitor\AppData\Local\Temp\ipykernel_74076\1188166265.py:1: DtypeWarning: Columns (68) have mixed types. Specify dtype option on import or set low_memory=False.
  df_listing = pd.read_csv('listings 2 reduced.csv')


In [ ]:
df_listing['host_is_superhost'] = df_listing['host_is_superhost'].map({'t': 1, 'f': 0})
df_listing['price'] = df_listing['price'].str.replace('$', '')
df_listing['price'] = df_listing['price'].str.replace(',', '')
df_listing['price'] = df_listing['price'].astype(float)
df_listing['price'] = df_listing['price'].fillna(0)


In [ ]:
# task1 maior diferença entre super host e não super host entre bairros
df_listing_avg = df_listing.groupby(['neighbourhood_cleansed', 'host_is_superhost'])['price'].median().unstack()
df_listing_avg['diff'] = df_listing_avg[1] - df_listing_avg[0]
df_listing_avg = df_listing_avg.sort_values(by='diff', ascending=False)
print('task1')
print(df_listing_avg.index[0])

task1
Westminster


In [ ]:
# task2 maior correlação entre preço e reviews
corr = df_listing[['review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'price']].corr(method= 'spearman')
corr = abs(corr)
corr = corr['price'].sort_values(ascending=False)
print('task2')
print(corr.index[1])


'review_scores_location'

In [ ]:
# task3 what is the average price difference between a professional host and a non-professional one? Consider a host as professional if they listings in more than 5 different locations (location is defined by the 'neighbourhood_cleansed' column).

df_listing_prof = df_listing.groupby('host_id')['neighbourhood_cleansed'].nunique()
df_listing_prof = df_listing_prof[df_listing_prof > 5]
df_listing_prof = df_listing_prof.to_frame().reset_index()
df_listing_prof = df_listing_prof.merge(df_listing, on='host_id')
df_listing_prof_avg = df_listing_prof.groupby('host_is_superhost')['price'].mean()
df_listing_prof_avg = abs(df_listing_prof_avg[1] - df_listing_prof_avg[0])
print('task3')
print(df_listing_prof_avg)




task3
106.61537933847524


In [ ]:
# task4 What is the median price premium given to entire homes / entire apartments with respect to other listings of the same neighborhood? Report the average across all neighborhoods. Use the 'room_type' column in the listings dataset to identify entire homes / entire apartments.


df_listing['room_type'] = df_listing['room_type'].map({'Entire home/apt': 'Entire home/apt', 'Private room': 'Private room', 'Shared room': 'Shared room', 'Hotel room': 'Hotel room'})
df_listing_room = df_listing.groupby(['neighbourhood_cleansed', 'room_type'])['price'].median().unstack()
df_listing_room['diff'] = abs(df_listing_room['Entire home/apt'] - df_listing_room['Private room'] - df_listing_room['Shared room'] - df_listing_room['Hotel room'])
df_listing_room_avg = df_listing_room['diff'].mean()
print('task4')
print(df_listing_room_avg)




task4
127.33333333333333


In [ ]:
# task5 What is the listing with the best expected revenue based on the last 12 mounths, considering 60% of guests leave reviews and every guest will stay only the minimum number of nights? Use both the 'listing' and 'reviews' datasets to answer this question and only use listing with minimum nights of stay <= 7. The minimum_nights column indicates the required minimum number of nights of stay for any listing
df_listing_minimum = df_listing[df_listing['minimum_nights'] <= 7]
df_review['date'] = pd.to_datetime(df_review['date'])
max_date = df_review['date'].max()
one_year_ago = max_date - pd.DateOffset(months=12)
df_last_12_months = df_review[df_review['date'] >= one_year_ago]

df_last_12_months = df_last_12_months.groupby('listing_id').size().reset_index(name='num_reviews')
df_last_12_months['estimated_bookings'] = df_last_12_months['num_reviews'] / 0.6
listings = df_listing[['id', 'price', 'minimum_nights']]
listings = listings[listings['minimum_nights'] <= 7]
merged_df = listings.merge(df_last_12_months, left_on='id', right_on='listing_id', how='inner')
merged_df['expected_revenue'] = merged_df['estimated_bookings'] * merged_df['minimum_nights'] * merged_df['price']
best_listing = merged_df.loc[merged_df['expected_revenue'].idxmax()]
print('task5')
print(best_listing['id'])



6611450.0


In [94]:
# task 6 What is the average difference between review scores of superhosts vc normal hosts? Use the 'review_scores_rating' column for determining the average review scores
df_listing_super = df_listing.groupby('host_is_superhost')['review_scores_rating'].mean()
df_listing_super_avg = abs(df_listing_super[1] - df_listing_super[0])
print('task6')
print(df_listing_super_avg)


task6
0.32333493329991114


In [ ]:
# task 7 Which host attribute has the second-strogest correlation with the number of reviews of the list? Use the following columns as the host attibutes: 'host_since', 'host_listing_count', 'host_identity_verified', 'calculated_host_listings_count', 'host_is_superhost', 'host_total_listings_count', 'host_is_superhost'. Use the 'number_of_reviews' as the column to find correlation with.
df_hosts_attributes = df_listing[['host_since', 'host_listings_count', 'host_identity_verified', 'calculated_host_listings_count', 'host_is_superhost', 'host_total_listings_count', 'number_of_reviews']]
df_hosts_attributes['host_since'] = pd.to_datetime(df_hosts_attributes['host_since'])
df_hosts_attributes['host_since'] = (pd.Timestamp.today() - df_hosts_attributes['host_since']).dt.days
df_hosts_attributes['host_identity_verified'] = df_hosts_attributes['host_identity_verified'].map({'t': 1, 'f': 0})
corr = df_hosts_attributes.corr(method='spearman')
corr = corr['number_of_reviews'].sort_values(ascending=False)
print('task7')
print(corr.index[2])

host_identity_verified


C:\Users\vitor\AppData\Local\Temp\ipykernel_74076\3832291567.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_hosts_attributes['host_since'] = pd.to_datetime(df_hosts_attributes['host_since'])
C:\Users\vitor\AppData\Local\Temp\ipykernel_74076\3832291567.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_hosts_attributes['host_since'] = (pd.Timestamp.today() - df_hosts_attributes['host_since']).dt.days
C:\Users\vitor\AppData\Local\Temp\ipykernel_74076\3832291567.py:5: SettingWithCopyWarning: 
A val